In [1]:
clear

In [2]:
import os
os.chdir("/Users/chris/Desktop/Dropbox/MasterThesisAFM/Coding")
os.getcwd()

'/Users/chris/Desktop/Dropbox/MasterThesisAFM/Coding'

In [3]:
import numpy as np
import random as rd
import networkx as nx
import csv

In [4]:
#beginning balances
bA1  = 135
bA2  = 145
bA3  = 90
bA4  = 30
bA5  = 215
bA6  = 70
bA7  = -75
bA8  = -50
bA9  = -60
bA10  = -120
bA11  = 0
bA12  = 0
bA13  = 0
bA14  = 0
begbal = (bA1, bA2, bA3, bA4, bA5, bA6, bA7, bA8, bA9, bA10, bA11, bA12, bA13, bA14)
beg = np.array(begbal).reshape(14,1)
beg_int32 = beg.astype(np.int32)
beg_int32.dtype

dtype('int32')

In [5]:
A11_A2 = abs(int(rd.normalvariate(270, 25)))  #recognize sales
A2_A1 = abs(int(rd.normalvariate(180, 20)))   #collect A/R
A7_A3 = abs(int(rd.normalvariate(115, 20)))   #buy inventory on account
A1_A3 = abs(int(rd.normalvariate(30, 10)))    #buy inventory for cash
A3_A12 = int(A11_A2*((rd.randrange(55,75,1))/100))  #recognize CoS (randomly 60-70% of Sales)
A4_A8 = abs(int(rd.normalvariate(30, 10)))    #net prepaid expenses with accrued liabilities
A7_A5 = abs(int(rd.normalvariate(20, 5)))     #buy PPE on account
A5_A13 = abs(int(rd.normalvariate(25, 10)))   #depreciate PPE
A6_A13 = abs(int(rd.normalvariate(10, 10)))   #amortize intangibles
A1_A7 = abs(int(rd.normalvariate(60, 10)))    #redeem accounts payable
A9_A13 = abs(int(rd.normalvariate(15, 10)))   #add provisions
A1_A10 = abs(int(rd.normalvariate(30, 10)))   #pay off LT debt
A10_A1 = abs(int(rd.normalvariate(50, 10)))   #take on LT debt
A8_A14 = int((A11_A2 - A3_A12 - A5_A13 - A6_A13 - A9_A13)*0.2)   #recognize tax expense
A1_A8 = abs(int(rd.normalvariate(25, 10)))    #redeem accrued liabilities

In [6]:
#create matrix from randomized transactions
transactions = (A11_A2, A2_A1, A7_A3, A1_A3, A3_A12, A4_A8, A7_A5, A5_A13, A6_A13, A1_A7, A9_A13, A1_A10, A10_A1, A8_A14, A1_A8)
ysim = np.array(transactions).reshape(15,1)
print(ysim)

[[226]
 [212]
 [106]
 [ 27]
 [137]
 [ 44]
 [ 15]
 [ 21]
 [ 10]
 [ 65]
 [ 18]
 [ 34]
 [ 50]
 [  8]
 [ 20]]


In [7]:
#produce list of nodes and edges for incidence matrix
#nodelist
def reader(filename):
    for (lineno, line) in enumerate(open(filename)):
        if lineno > 0: # skip header
            yield line

filename = "case1_nodes.csv"
(l_nodes, nodenames, beg, end, change) = zip(*( row for row in csv.reader(reader(filename))))


#edgelist
def reader(filename2):
    for (lineno, line) in enumerate(open(filename2)):
        if lineno > 0: # skip header
            yield line

filename2 = "case1_edges.csv"
(from_, to_, transactions_) = zip(*( row for row in csv.reader(reader(filename2))))

l_edges = list(zip(from_, to_))

In [8]:
#produce incidence matrix
def incidence_matrix(A, nodelist=None, edgelist=None, 
                     oriented=True, weight=None):
    if nodelist is None:
        nodelist = l_nodes
    if edgelist is None:
        edgelist = l_edges
    A = np.zeros((len(nodelist),len(edgelist)))
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        if u == v: continue  # self loops give zero column
        try:
            ui = node_index[u]
            vi = node_index[v]
        except KeyError:
            raise NetworkXError('node %s or %s in edgelist '
                                'but not in nodelist"%(u,v)')
        if weight is None:
            wt = 1
        else:
            if DG.is_multigraph():
                ekey = e[2]
                wt = DG[u][v][ekey].get(weight,1)
            else:
                wt = DG[u][v].get(weight,1)
        if oriented:
            A[ui,ei] = -wt
            A[vi,ei] = wt
        else:
            A[ui,ei] = wt
            A[vi,ei] = wt
    return np.asmatrix(A)

In [9]:
xsim = incidence_matrix("A") * ysim
xsim_int32 = xsim.astype(np.int32)
print(xsim_int32)

[[ 116]
 [  14]
 [  -4]
 [ -44]
 [  -6]
 [ -10]
 [ -56]
 [  56]
 [ -18]
 [ -16]
 [-226]
 [ 137]
 [  49]
 [   8]]


In [10]:
endsim = beg_int32 + xsim_int32
print(endsim)

[[ 251]
 [ 159]
 [  86]
 [ -14]
 [ 209]
 [  60]
 [-131]
 [   6]
 [ -78]
 [-136]
 [-226]
 [ 137]
 [  49]
 [   8]]


In [11]:
#redefine arc capacities
d_nodes = dict(zip(l_nodes, xsim))

In [12]:
#create DiGraph = DG
DG=nx.DiGraph()
#ADD SOURCE/SINK EDGES, by definition all nodes are included in edgelist
for k, v in d_nodes.items():
    
    if int(v)<0:
        DG.add_edge('S',k)
        nx.set_edge_attributes(DG, 'capacity', {('S',k): abs(int(v))})
        print("from 'S' to", k, "with capacity =",abs(int(v)))

    if int(v)>0:
        DG.add_edge(k,"T")
        nx.set_edge_attributes(DG, 'capacity', {(k, 'T'): abs(int(v))})
        print("from", k,"to 'T'","with capacity =",int(v))

from A1 to 'T' with capacity = 116
from A2 to 'T' with capacity = 14
from 'S' to A3 with capacity = 4
from 'S' to A4 with capacity = 44
from 'S' to A5 with capacity = 6
from 'S' to A6 with capacity = 10
from 'S' to A7 with capacity = 56
from A8 to 'T' with capacity = 56
from 'S' to A9 with capacity = 18
from 'S' to A10 with capacity = 16
from 'S' to A11 with capacity = 226
from A12 to 'T' with capacity = 137
from A13 to 'T' with capacity = 49
from A14 to 'T' with capacity = 8


In [13]:
#add transaction edges
for f,t in l_edges:
    DG.add_edge(f,t)
    DG.edge[f][t]['capacity'] = 99999

In [14]:
print('no. of nodes:',DG.number_of_nodes())
print('no. of edges:',DG.number_of_edges())
print('maximum flow:',nx.maximum_flow_value(DG, s='S', t='T'))
print('minimum cut: ',nx.minimum_cut_value(DG, s="S", t="T"))

no. of nodes: 16
no. of edges: 29
maximum flow: 380
minimum cut:  380


In [15]:
#run max-flow min cut algorithm
data = nx.maximum_flow(DG, s="S", t="T")
nx.maximum_flow(DG, s="S", t="T")

(380,
 {'A1': {'A10': 0, 'A3': 77, 'A7': 15, 'A8': 20, 'T': 116},
  'A10': {'A1': 16},
  'A11': {'A2': 226},
  'A12': {'T': 137},
  'A13': {'T': 49},
  'A14': {'T': 8},
  'A2': {'A1': 212, 'T': 14},
  'A3': {'A12': 137},
  'A4': {'A8': 44},
  'A5': {'A13': 21},
  'A6': {'A13': 10},
  'A7': {'A3': 56, 'A5': 15},
  'A8': {'A14': 8, 'T': 56},
  'A9': {'A13': 18},
  'S': {'A10': 16,
   'A11': 226,
   'A3': 4,
   'A4': 44,
   'A5': 6,
   'A6': 10,
   'A7': 56,
   'A9': 18},
  'T': {}})

In [16]:
#define y from max-flow min cut
y1 = []
for x, z in l_edges:
    try:
        y1.append(data[1][x][z])
    except:
        pass
y_alg = np.array(y1).reshape(len(l_edges),1)

In [17]:
y_alg - ysim

array([[  0],
       [  0],
       [-50],
       [ 50],
       [  0],
       [  0],
       [  0],
       [  0],
       [  0],
       [-50],
       [  0],
       [-34],
       [-34],
       [  0],
       [  0]])